**Train on whole dataset and test on test set**

In [33]:
import os
import pandas as pd 
import numpy as np

from expert_tree import get_expert_tree_results, Expert_Tree
from wrapper import best_first_search_mg

from sklearn.metrics import confusion_matrix

In [2]:
test_set_dir = 'avon_expert_tree.csv'
data_test = pd.read_csv(test_set_dir)
#data_test.head(5)
print(data_test.shape)

(50, 14)


In [34]:

# fill 0 for missing data
for column in ['Mobility', 'ArmSwelling', 'BreastSwelling', 'Skin', 'ChestWallSwelling']:
    data_test[column].fillna(0, inplace=True)
# drop data missing important feature
data_test = data_test.dropna(subset=['TIME_LAPSE', 'LVC'])
# delete Username
data_test = data_test.drop(columns='Username')
# convert string to float
#data_test = data_test.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data_test = data_test.astype(np.float64)
# drop rows containing nan
data_test = data_test.dropna()
# drop data within a 6 months
data_test = data_test[data_test.TIME_LAPSE >=0.5]
# add log of time elapsed
data_test['TIME_LAPSE_LOG'] = np.log(data_test['TIME_LAPSE'])

print('shape:')
print(data_test.shape)

shape:
(50, 16)


In [35]:
# generate 3 labels
Y_test, _, _ = get_expert_tree_results(data_test, False,3)
# Y_TEST is expert tree results

In [36]:
print(Y_test)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [46]:
data_test2 = pd.read_csv(test_set_dir)

In [47]:
truth_labels =data_test2['LE_stage']
temp = truth_labels
print(temp)

0     4
1     4
2     3
3     3
4     4
5     4
6     2
7     4
8     2
9     3
10    4
11    4
12    4
13    3
14    4
15    3
16    4
17    4
18    3
19    3
20    2
21    4
22    2
23    3
24    3
25    4
26    3
27    3
28    3
29    3
30    3
31    4
32    1
33    4
34    3
35    4
36    3
37    1
38    4
39    3
40    3
41    3
42    2
43    3
44    3
45    4
46    4
47    4
48    2
49    4
Name: LE_stage, dtype: int64


In [48]:
truth_labels.replace(1,0, True)
truth_labels.replace(2,1,True)
truth_labels.replace(3,2, True) #modereate and severe to tag 2 
truth_labels.replace(4,2, True)

In [49]:
truth_labels


0     2
1     2
2     2
3     2
4     2
5     2
6     1
7     2
8     1
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
20    1
21    2
22    1
23    2
24    2
25    2
26    2
27    2
28    2
29    2
30    2
31    2
32    0
33    2
34    2
35    2
36    2
37    0
38    2
39    2
40    2
41    2
42    1
43    2
44    2
45    2
46    2
47    2
48    1
49    2
Name: LE_stage, dtype: int64

EVALUATE THE RESULTS

In [17]:
from sklearn.metrics import confusion_matrix


In [37]:
""""
New expert tree that was based on the changes made on 4.28.2020 with Dr Fu
"""

SPECIAL=  -1
non_LE= 0
Latent = 1
Mild = 2
Moderate = 3
Severe = 4

#update: 3.3.2020 00.26 am
# code with all 12 fixed to original thesis model,everything in that drawing
class Expert_Tree2:
    def __init__(self, row ):
        #self.patient_data = row
        self.d = dict()
        self.d['mobility']= row[0]
        self.d['AS']= row[1]
        self.d['BS']= row[2]
        self.d['skin']= row[3]
        self.d['pas']= row[4]
        self.d['fht']= row[5]
        self.d['discomfort']= row[6]
        self.d['symcount']= row[7]
        self.d['timelap']= row[8]
        ##??? fluid question
        #self.d['fluid'] = max(self.d['AS'], self.d['BS'],self.d['pas'],self.d['fht'],self.d['skin'] )#???
        self.d['fluid'] = row[11]
        self.d['LVC'] = row[9]
        self.d['CS'] =row[10]
       # self.d['fluid'] = row[11]
        self.str = ''
    def run(self):
        if self.d['LVC'] <=0.05:
            if self.d['AS'] ==0:
                if self.d['BS'] ==0:
                    if self.d['skin']>1:
                        if self.d['symcount'] < 6:
                            self.str +='a'
                            return Latent
                        else:
                            self.str +='b'
                            return Mild
                    else: #skin <=1
                        # *************** 12 month change
                        if self.d['CS']>1 and self.d['timelap']>=1: #check month
                            self.str +='c'
                            return Mild
                        elif self.d['CS']<=1:
                            if self.d['fht'] ==0:
                                if self.d['discomfort']>=1 and self.d['timelap']<=0.5:
                                    self.str +='d'
                                    return Latent
                                if (self.d['mobility']>=1 or self.d['pas']>=1)and self.d['timelap']<=0.5:
                                    self.str +='e'
                                    return Latent
                                
                            elif self.d['fht']>=1:
                                # *************** 12 month change
                                if self.d['discomfort']>1 and self.d['timelap']>=1:
                                    self.str +='f'
                                    return Mild
                                if self.d['mobility']>=1 and self.d['timelap']<=0.5:
                                    self.str +='g'
                                    return Mild
                            
                                if self.d['fht']==1 and self.d['pas']>1 and self.d['timelap']>0.5:
                                    self.str +='h'
                                    return Mild
                            # None of above
                            # *************** 12 month change
                            if self.d['CS']==1 and self.d['timelap'] < 1:
                                self.str +='i'
                                return Latent

                elif self.d['BS'] ==1:
                    if self.d['skin'] > 1 and self.d['fht']>0:
                        self.str +='j'
                        return Moderate
                    elif self.d['skin'] <=1 or  self.d['fht']==0 : ###ask?
                        # *************** 12 month change
                        if self.d['CS']>1 and self.d['timelap'] >=1:
                            self.str +='k'
                            return Mild
                        elif self.d['CS']<=1: ##ask
                            # *************** 12 month change
                            if self.d['discomfort']>1 and self.d['timelap']>=1:
                                self.str +='l'
                                return Mild
                            elif self.d['mobility'] >=1 and self.d['timelap'] < 1: #new change here
                                self.str +='m'
                                return Mild
                            #none of above
                            # *************** 12 month change
                            if self.d['timelap'] <1: ##ask:
                                self.str +='n'
                                return Latent

                else: #BS>1
                    if self.d['skin'] > 1:
                        if self.d['timelap'] <=0.5:
                            self.str +='o'
                            return Mild
                        else: #timelap>6 months
                            self.str +='p'
                            return Moderate
                    else: #skin<=1
                        #below code is what i added
                        # *************** 12 month change
                        if self.d['timelap'] >=1:
                            self.str +='q'
                            return Mild
                        elif self.d['mobility'] >=1 and self.d['timelap'] <=0.5: ##ask? mobility?
                            self.str +='r'
                            return Mild
                         #^^^^ timlap<=6 months condition removed:

                        '''

                        if self.d['mobility'] >=1:# and self.d['timelap'] <=0.5: ##ask? mobility?
                            self.str +='q'
                            return Mild
                        '''
                        '''
                        #code with 12 month fixed to 6 month
                        if self.d['timelap'] >=0.5:
                            self.str +='q'
                            return Mild
                        elif self.d['mobility'] >=1:# and self.d['timelap'] <=0.5: ##ask? mobility?
                            self.str +='r'
                            return Mild
                        '''

            elif self.d['AS'] ==1:    
                if self.d['skin'] ==0:
                    self.str +='s'
                    return Mild
                elif self.d['skin'] ==1:
                    self.str +='t'
                    return Moderate
                elif self.d['skin']>1 and self.d['fht']>0 and self.d['pas']>0 and self.d['mobility']>0:
                    if  8 <= self.d['symcount'] and self.d['symcount'] <= 12:
                        self.str +='u'
                        return Moderate
                    elif self.d['symcount'] >12:
                        self.str +='v'
                        return Severe


            elif self.d['AS'] >1: 
                if self.d['skin'] >1 and self.d['fht'] >0 and self.d['pas']>0 and self.d['mobility'] >0 and self.d['symcount'] >12:
                    # a bit of change 3.3.2020 1.02 am
                    self.str +='w'
                    return Severe
                    #if self.d['symcount'] >12:
                    #    self.str +='w'
                    #    return Severe
                else: # a bit of change
                    self.str +='x'
                    return Moderate

            #  else:  NONE OF ABOVE
            #still check here 
            if self.d['LVC'] >0.03:
                # *************** 12 month change
                if self.d['symcount'] >6 and 0.5 < self.d['timelap']:
                    self.str +='y'
                    return Mild
                else:
                    self.str +='z'
                    return Latent
            else: #LVC <= 0.03
                if self.d['symcount']>6 and self.d['fluid']>=2:
                    self.str +='A'
                    return Latent
                else:
                    self.str +='B'
                    return non_LE

        else: #LVC >=0.05
            # AS ==0 
            if self.d['AS']==0:
                    if self.d['CS']>1 and (self.d['fht'] >0 or self.d['pas']>0):
                        self.str +='C'
                        return Severe
                    else:
                        # BS ==0 
                        if self.d['BS'] ==0:
                            if self.d['symcount'] <4:
                                self.str +='D'
                                return non_LE
                            elif 4 <= self.d['symcount'] and self.d['symcount'] <6:
                                self.str +='E'
                                return Latent
                            elif 6 <= self.d['symcount'] and self.d['symcount'] <10:
                                if self.d['timelap']<= 0.5:
                                    self.str +='F'
                                    return Latent
                                else:
                                    self.str +='G'
                                    return Mild
                            else: #symp_count >=10
                                self.str += 'H'
                                return Moderate
                        # BS >= 1
                        elif self.d['BS'] >=1:
                            if self.d['mobility']>1:
                                self.str +='I'
                                return Moderate
                            elif self.d['skin']>1:
                                self.str +='J'
                                return Moderate
                            elif self.d['discomfort']>1:
                                self.str +='K'
                                return Moderate
                            #none of Severe above
                            if self.d['BS'] > 1: 
                                self.str +='L'
                                return Moderate
                            if self.d['mobility']==1:
                                # *************** 12 month change
                                if  self.d['timelap'] <=0.5:
                                    self.str +='M'
                                    return Latent
                                elif self.d['timelap'] >0.5 :
                                    self.str +='N'
                                    return Mild
                                
                        if self.d['CS']>1 and self.d['fht']>0 and self.d['pas']==0:
                            #maybe ask this condition?
                            
                            self.str +='O'
                            return Moderate

            elif self.d['AS'] ==1:
                # is this a division?
                if (self.d['fht'] >1 or self.d['pas'] >1) and self.d['BS'] >0: 
                    self.str +='P'
                    return Severe
                elif self.d['mobility']>1:
                    self.str +='Q'
                    return Severe
                elif self.d['skin']>1:
                    self.str +='R'
                    return Severe
                elif self.d['discomfort'] >1:
                    self.str +='S'
                    return Severe
                #if none of above
                if self.d['mobility'] ==1 and self.d['timelap'] >0.5 :
                    self.str +='T'
                    return Moderate
                if (self.d['fht'] >=1 or self.d['pas'] >=1) and self.d['timelap'] >0.5 :
                    self.str +='U'
                    return Moderate
            elif self.d['AS']>1:
                self.str +='V'
                return Severe
            self.str +='W'
            #if none above
            return Mild
        self.str +='X'
        return SPECIAL
        #return non_LE






In [38]:

def get_expert_tree_results2(data, class_number):
    flags = []
    # data_lst = list(X_arr)
    # row,col = data.shape[0], data.shape[1]
    # print((row,col))
    # print(len(LVC_lst))
    expert_tree_result = []
    expert_truth_labels = []

    for i, row in data.iterrows():  # row = 101
        # in row
        #         row_lst = []
        #         for c in range(0,col):
        #             row_lst.append(data[r][c])

        # run the algorithm in the expert tree
        exp_tree = Expert_Tree2(row)
        expert_tag = exp_tree.run()

        """
        non_LE= 0
        Latent = 1
        Mild = 2
        Moderate = 3
        Severe = 4

        expand expert_truth_labels_accordingly
        """
        if expert_tag == 0:
            expert_truth_labels.append('Non_LE')
        elif expert_tag == 1:
            expert_truth_labels.append('Latent')
        elif expert_tag == 2:
            expert_truth_labels.append('Mild')
        elif expert_tag == 3:
            expert_truth_labels.append('Moderate')
        elif expert_tag == 4:
            expert_truth_labels.append('Severe')
        else:
            expert_truth_labels.append('ERROR!')

        if class_number == 3:
            if expert_tag == 1 or expert_tag == 0:  # none or latent
                result = 0
            elif expert_tag == 2:  # mild
                result = 1
            elif expert_tag > 2:  # moderate severe
                result = 2
            else:
                result = -999
        elif class_number == 2:  # if not tree clas (if it two-class)
            if expert_tag == 1 or expert_tag == 0:  # none or latent
                result = 0
            elif expert_tag > 1:  # mild moderate severe
                result = 1
            else:
                result = -999
        elif class_number == 4:  # class number 4
            if expert_tag == 1 or expert_tag == 0:  # none or latent
                result = 0
            elif expert_tag == 2:  # mild
                result = 1
            elif expert_tag == 3:  # moderate
                result = 3
            elif expert_tag == 4:  # severe
                result = 4
            else:
                result = -999
        else:
            print('invalid class number')

        expert_tree_result.append(result)
        flags.append(exp_tree.str)

    return expert_tree_result, expert_truth_labels, flags


In [41]:
 expert_tree_result2, expert_truth_labels, flags = get_expert_tree_results2(data_test, 3)

In [45]:
expert_tree_result2

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [24]:
t = confusion_matrix(truth_labels, Y_test, labels=[0,1,2]).ravel()
t = confusion_matrix(truth_labels, Y_test, labels=[0,1,2])
'''
print(tn, fp, fn, tp)
validation_accuracy = (tn+tp)/(tn+tp+fn+fp)
sensitivity = tp/(fn+tp)
specificty= tn/(tn+fp)
print(validation_accuracy, sensitivity,specificty )
'''
t

array([[ 2,  0,  0],
       [ 0,  1,  5],
       [ 0,  0, 42]], dtype=int64)

In [16]:
len(truth_labels) == len(Y_test)

True

In [13]:
#def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
#def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
#def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
#def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]

In [50]:
data_test2['truth_labels'] = truth_labels
data_test2['Y_test'] = Y_test

In [52]:
data_test2['expert_tree_old'] = expert_tree_result2

In [53]:
data_test2[]

,Username,Mobility,ArmSwelling,BreastSwelling,Skin,PAS,FHT,DISCOMFORT,SYM_COUNT,TIME_LAPSE,LVC,ChestWallSwelling,fluid_total,LE_stage,truth_labels,Y_test,expert_tree_old
0,C001*,4,4,4,4,4,4,4,20,3,0.540256,4,6,2,2,2,2
1,C002*,1,4,0,1,1,3,1,11,15,0.254824,0,4,2,2,2,2
2,C003,0,2,1,1,1,3,1,9,4,0.110934,0,5,2,2,2,2
3,C004,1,3,0,1,3,1,2,12,2,0.079574,0,4,2,2,2,2
4,C005,0,4,0,1,0,3,3,11,17,0.283379,3,4,2,2,2,2
5,C006*,1,4,0,2,3,4,2,14,19,0.359089,2,5,2,2,2,2
6,C007*,0,1,0,0,0,2,1,6,3,0.066416,0,2,1,1,2,2
7,C008,2,4,3,2,1,3,3,15,14,0.599020,3,6,2,2,2,2
8,C009,2,2,1,0,1,1,2,12,4,0.083401,1,5,1,1,2,2
9,C010*,1,2,0,0,1,1,1,10,1,0.107438,0,3,2,2,2,2


In [ ]:
for i in range()

In [7]:
# drop LVC and time time elapse
data_test = data_test.drop(columns=['LVC', 'TIME_LAPSE', 'fluid_total'])
# prent shape and columns
print(f"data shape: {data_test.shape}")
print("columns:")
print(data_test.columns.values)

data shape: (176, 20)
columns:
['Mobility' 'ArmSwelling' 'BreastSwelling' 'Skin' 'PAS' 'FHT' 'DISCOMFORT'
 'SYM_COUNT' 'ChestWallSwelling' 'Chemotherapy' 'Radiation' 'Age'
 'SLNB_Removed_LN' 'ALND_Removed_LN' 'SLNB_ALND_Removed' 'Mastectomy'
 'Lumpectomy' 'Hormonal' 'BMI' 'TIME_LAPSE_LOG']


In [8]:
# get features for model  
X_test = data_test[feats].values
print(f"X shape: {X_test.shape}")
print(f"y length: {len(Y_test)}")

X shape: (176, 12)
y length: 176


Training set

In [9]:
# define which dataset to use 
IF_USE_PRE_DATA = 1

delete_rows = ['A001', 'A003', 'A004', 'A005', 'A015', 'A016',
               'A018', 'A025', 'A026', 'A029', 'A031', 'A032',
               'A035', 'A036', 'A038', 'A042', 'A046', 'A055',
               'T010', 'T013', 'T014', 'T016', 'T018', 'T019',
               'T021', 'T026', 'T029', 'T030', 'T036', 'T038',
               'T040', 'T043', 'T044', 'T054', 'T055', 'T056',
               'T057'] 

# read in dataset 
DATA_PATH = ('./data/META_data_correct_v5.csv', './data/feature_selection_preprocessed_data.csv')[IF_USE_PRE_DATA > 0]
data = pd.read_csv(DATA_PATH)
print(f'data shape: {data.shape}')
data = data[data.Username != 'ML380']

# drop list of subjects
data = data[~data.Username.isin(delete_rows)]
# drop data within a 6 months
data = data[data.TIME_LAPSE >=0.5]
data = data.drop(columns=['Username', 'Unnamed: 0'] if IF_USE_PRE_DATA else 'Username')

# the data_pre contain '#DEV/0!' or "" to make the coloumn string instead of float
# and these dirty element would cause error in .astype operation
# here convert "" or  '#DEV/0!' to nan
#data1 = data
data = data.apply(lambda x: pd.to_numeric(x, errors='coerce'))
# drop rows containing nan
data = data.dropna()

# add log of time elapsed
data['TIME_LAPSE_LOG'] = np.log(data['TIME_LAPSE'])

# prent shape and columns
print(f"data shape: {data.shape}")
print("columns:")
print(data.columns.values)

data shape: (906, 25)
data shape: (858, 24)
columns:
['Mobility' 'ArmSwelling' 'BreastSwelling' 'Skin' 'PAS' 'FHT' 'DISCOMFORT'
 'SYM_COUNT' 'TIME_LAPSE' 'LVC' 'ChestWallSwelling' 'fluid_total'
 'Chemotherapy' 'Radiation' 'Age' 'SLNB_Removed_LN' 'ALND_Removed_LN'
 'SLNB_ALND_Removed' 'Number_nodes' 'Mastectomy' 'Lumpectomy' 'Hormonal'
 'BMI' 'TIME_LAPSE_LOG']


In [10]:
# generate 3 labels
Y_train, _, _ = get_expert_tree_results(data, three_class=True)

In [11]:
# drop LVC and time time elapse
data = data.drop(columns=['LVC', 'TIME_LAPSE', 'fluid_total', 'Number_nodes'])
# prent shape and columns
print(f"data shape: {data.shape}")
print("columns:")
print(data.columns.values)
# get features for model  
X_train = data[feats].values
print(f"X shape: {X_train.shape}")
print(f"y length: {len(Y_train)}")

data shape: (858, 20)
columns:
['Mobility' 'ArmSwelling' 'BreastSwelling' 'Skin' 'PAS' 'FHT' 'DISCOMFORT'
 'SYM_COUNT' 'ChestWallSwelling' 'Chemotherapy' 'Radiation' 'Age'
 'SLNB_Removed_LN' 'ALND_Removed_LN' 'SLNB_ALND_Removed' 'Mastectomy'
 'Lumpectomy' 'Hormonal' 'BMI' 'TIME_LAPSE_LOG']
X shape: (858, 12)
y length: 858


Classifier

In [12]:
# gradient boosting tree
from sklearn.ensemble import GradientBoostingClassifier
params = {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 70}
gbt = GradientBoostingClassifier(**params)
gbt

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=70,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

**Train and test using feature subset**

In [13]:
print(f'X train: {X_train.shape}')
print(f'Y train: {len(Y_train)}')
print(f'X test: {X_test.shape}')
print(f'Y test: {len(Y_test)}')

X train: (858, 12)
Y train: 858
X test: (176, 12)
Y test: 176


In [14]:
gbt= gbt.fit(X_train, Y_train)
feature_weight_pair = sorted(zip(feats, gbt.feature_importances_), key=lambda pair : pair[1], reverse=True)
print('learnt importance:')
feature_weight_pair

learnt importance:


[('ArmSwelling', 0.6279334980499687),
 ('FHT', 0.12260146087839063),
 ('BreastSwelling', 0.06772541134456962),
 ('Skin', 0.0551889442155744),
 ('DISCOMFORT', 0.0445775593357751),
 ('TIME_LAPSE_LOG', 0.02975591672586188),
 ('Mobility', 0.021879943363565395),
 ('PAS', 0.01777250559760038),
 ('BMI', 0.008542573995460687),
 ('ChestWallSwelling', 0.004022186493233382),
 ('Chemotherapy', 0.0),
 ('Mastectomy', 0.0)]

In [15]:
accu = gbt.score(X_test, Y_test)
print(f'accuracy: {accu}')
CM = confusion_matrix(Y_test, gbt.predict(X_test))
print('confusion matrix:')
print(CM)

accuracy: 0.9375
confusion matrix:
[[122   1   0]
 [  5  22   0]
 [  0   5  21]]


**Train and test using all feature**

In [16]:
X_train = data.values
X_test = data_test.values
print(f'X train: {X_train.shape}')
print(f'Y train: {len(Y_train)}')
print(f'X test: {X_test.shape}')
print(f'Y test: {len(Y_test)}')

X train: (858, 20)
Y train: 858
X test: (176, 20)
Y test: 176


In [17]:
gbt= gbt.fit(X_train, Y_train)
feature_weight_pair = sorted(zip(data.columns.values, gbt.feature_importances_), key=lambda pair : pair[1], reverse=True)
print('learnt importance:')
feature_weight_pair

learnt importance:


[('ArmSwelling', 0.6019545472027077),
 ('SYM_COUNT', 0.15478052941434894),
 ('FHT', 0.05922034058589855),
 ('BreastSwelling', 0.055728154259867754),
 ('Skin', 0.04116786198650627),
 ('DISCOMFORT', 0.028057565913575364),
 ('TIME_LAPSE_LOG', 0.025118394143512565),
 ('Mobility', 0.010101035177408848),
 ('BMI', 0.006985547324563689),
 ('PAS', 0.00416003969746289),
 ('Age', 0.0030952727500087947),
 ('SLNB_ALND_Removed', 0.0028694617157935054),
 ('ChestWallSwelling', 0.0025479861812716755),
 ('ALND_Removed_LN', 0.0022367150137939888),
 ('SLNB_Removed_LN', 0.0012148865912766525),
 ('Radiation', 0.00033425003740272433),
 ('Hormonal', 0.00024222614321694515),
 ('Lumpectomy', 0.0001851858613829529),
 ('Chemotherapy', 0.0),
 ('Mastectomy', 0.0)]

In [18]:
accu = gbt.score(X_test, Y_test)
print(f'accuracy: {accu}')
CM = confusion_matrix(Y_test, gbt.predict(X_test))
print('confusion matrix:')
print(CM)

accuracy: 0.9261363636363636
confusion matrix:
[[122   1   0]
 [  6  20   1]
 [  0   5  21]]


Check if there are sample overlap between the train and validation:

In [19]:
dataTest = pd.read_csv(test_set_dir)
dataTrain = pd.read_csv(DATA_PATH)

In [20]:
s1 = pd.merge(dataTest, dataTrain, how='inner', on=['Username'])

In [21]:
s1

,Username,Mobility_x,ArmSwelling_x,BreastSwelling_x,Skin_x,PAS_x,FHT_x,DISCOMFORT_x,SYM_COUNT_x,TIME_LAPSE_x,...,Radiation_y,Age_y,SLNB_Removed_LN_y,ALND_Removed_LN_y,SLNB_ALND_Removed_y,Number_nodes,Mastectomy_y,Lumpectomy_y,Hormonal_y,BMI_y
0,ML380,2,0,2,0,1,0,1,5,0.8,...,1,70.0,1.0,0.0,0.0,1.0,0,1,1,39.8


In [22]:
# print(sorted(dataTest['Username'].values))

In [23]:
# print(sorted(dataTrain['Username'].values))

Check label distribution

In [24]:
{label:count for label, count in zip(*np.unique(Y_test, return_counts=True))}

{0: 123, 1: 27, 2: 26}

In [25]:
{label:count for label, count in zip(*np.unique(Y_train, return_counts=True))}

{0: 385, 1: 219, 2: 254}